In [20]:
import numpy as np
import random
from timeit import default_timer as timer
from datetime import datetime

In [21]:
def generate_cities(file):
    file1 = open(file, 'r')
    start = False
    cities_dict = {}

    while True:

        # Get next line from file
        line = file1.readline()

        # if line is empty
        # end of file is reached
        if not line:
            break

        if start:
            citystr, xstr, ystr = line.rstrip().split()
            city = citystr
            x = float(xstr)
            y = float(ystr)
            cities_dict[city] = [x, y]

        if line == "NODE_COORD_SECTION\n":
            start = True
    file1.close()
    return cities_dict

In [22]:
# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

In [23]:
cities_dict = generate_cities("Uruguay734.tsp")
names_list = list(cities_dict.keys())
names_list = np.array(names_list)
mutation_rate = 0.5
n_population = 50
n_cities = 734

In [24]:
# First step: Create the first population set
def genesis(city_list, n_population):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
population_set

array([['445', '308', '274', ..., '564', '46', '128'],
       ['547', '419', '273', ..., '58', '293', '292'],
       ['137', '244', '613', ..., '508', '490', '693'],
       ...,
       ['341', '715', '169', ..., '570', '370', '500'],
       ['285', '271', '546', ..., '476', '691', '221'],
       ['99', '532', '174', ..., '205', '466', '153']], dtype='<U3')

In [25]:
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total

In [26]:
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([1644661.27286831, 1612152.10657685, 1647625.47806837,
       1590977.06308052, 1656571.59682174, 1652406.7100953 ,
       1649422.53737809, 1679364.7235403 , 1657548.67131056,
       1652488.4983897 , 1605241.471301  , 1611324.52968825,
       1591779.46547541, 1634401.87929078, 1643525.63285534,
       1667825.14918873, 1621641.52618296, 1664522.22428817,
       1646044.05578256, 1622735.13780247, 1640202.39936262,
       1639672.06499533, 1645934.92151666, 1639264.33005614,
       1643865.70190095, 1641422.07514473, 1617891.9324882 ,
       1604979.6585842 , 1604050.39917448, 1630564.57666238,
       1645895.56574093, 1651317.47665   , 1633049.62740962,
       1630866.18396179, 1632183.44009375, 1626006.65375458,
       1636589.17585571, 1640416.81195683, 1620456.43773072,
       1649227.52671954, 1627546.18047347, 1670527.03387215,
       1626663.26349853, 1617779.16949569, 1640688.45996731,
       1657091.94808766, 1605916.10986898, 1650581.55023733,
       1588182.60407303,

In [27]:
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)

array(['249', '689', '88', '631', '31', '382', '668', '341', '641', '446',
       '287', '565', '246', '10', '138', '227', '346', '413', '485',
       '221', '396', '582', '45', '142', '53', '522', '229', '67', '653',
       '327', '544', '38', '202', '275', '226', '443', '314', '170',
       '172', '511', '158', '466', '709', '261', '106', '187', '417',
       '174', '630', '462', '112', '128', '729', '5', '554', '390', '247',
       '345', '501', '566', '159', '463', '567', '655', '191', '397',
       '306', '422', '576', '178', '642', '380', '506', '119', '502',
       '72', '374', '435', '609', '492', '633', '662', '508', '589',
       '111', '55', '157', '239', '236', '207', '218', '495', '449', '7',
       '124', '676', '585', '448', '296', '680', '717', '25', '200',
       '366', '550', '486', '161', '457', '264', '414', '347', '467',
       '21', '659', '364', '445', '562', '360', '651', '415', '292',
       '177', '420', '351', '597', '733', '425', '209', '210', '726',
       

In [28]:
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring
            
    
    
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list)

In [29]:
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)

        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring
    
    
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)

In [31]:
best_solution = [-1,np.inf,np.array([])]
start = timer()
for i in range(500):
    if i%100==0: print(i, fitnes_list.min(), fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
    
    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list)
    
    mutated_pop = mutate_population(new_population_set)
end = timer()
print("Time: {}".format(end - start))

0 1571431.3115543886 1636655.271981449 15/12/22 14:51
100 1576686.5033700315 1632291.3996701443 15/12/22 14:52
200 1576546.4231065814 1631091.3322015407 15/12/22 14:52
300 1574644.6104999734 1636722.5903480244 15/12/22 14:53
400 1579007.3355248019 1636107.2192807132 15/12/22 14:54
Time: 223.46098479999995


In [32]:
best_solution[1]

1518658.5292124723